# MATH 405/607 

# Numerical Methods for Differential Equations

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[course page]](https://github.com/cortner/math405_2022)


## IVPs Part 3: Hamiltonian Systems

* Hamiltonian Systems, conservation of energy 
* Symplectic Euler methods
* Outlook: Symplectic integrators

In [ ]:
include("math405.jl")

euler_step(u, f, h) = u + h * f(u)

function implicit_euler_step(u, f, h; nnewton=5, 
                             df = u_ -> ForwardDiff.jacobian(f, u_))
    u1 = u
    for n = 1:nnewton # unew - uold - h * f(unew) = 0
        u1 -= (I - h * df(u1)) \ (u1 - u - h * f(u1))
    end        
    return u1 
end

function rk4_step(u, f, h)
    k1 = h * f(u)
    k2 = h * f(u + 0.5 * k1)
    k3 = h * f(u + 0.5 * k2)
    k4 = h * f(u + k3)
    return u + k1/6 + k2/3 + k3/3 + k4/6
end

euler_method(f, u0, h, Tf) = _iterate(f, u0, h, Tf, euler_step)

implicit_euler_method(f, u0, h, Tf; kwargs...) = _iterate(f, u0, h, Tf, implicit_euler_step; kwargs...)

function _iterate(f, u0, h, Tf, stepper; kwargs...)
    t = 0:h:Tf
    U = zeros(length(u0), length(t))
    U[:, 1] = u0
    for n = 2:length(t)
        U[:, n] = stepper(U[:,n-1], f, h; kwargs...)
    end 
    return U, t
end         

### N-Body Problem

Point particles at positions ${\bf r}_i$ with masses $m_i$
$$
    m_i \ddot{\bf r}_i = \sum_{j \neq i} \frac{G m_i m_j ({\bf r}_i - {\bf r}_j)}{|{\bf r}_i - {\bf r}_j|^3}
$$

In [ ]:
u2rv(u) = (n = length(u)÷4; (reshape(u[1:2*n], (2, n)), reshape(u[2*n+1:end], (2, n))))
rv2u(r, v) = [r v][:]

function nbody_accel(r, G, m)
    a = zeros(eltype(r), 2, n)
    for i = 1:n, j = 1:n
        i == j && continue
        r_ij = r[:,j] - r[:,i]
        a[:, i] += r_ij * G * m[j] / norm(r_ij)^3
    end
    return a 
end 

function nbody(u, G, m)
    r, v = u2rv(u)
    a = nbody_accel(r, G, m)
    return [ v a ][:]
end


|   Sun       |   Earth     |  Mercury   |   Mars       |   Venus      |
|-------------|-------------|------------|--------------|--------------|
| 1.989e30 kg | 5.972e24 kg | 3.30e23 kg | 6.4219e23 kg | 4.869e24 kg  |
|   0 AU      |   1 AU      |  0.38 AU   |   1.52 AU    |   0.72 AU    |


In [ ]:
# Masses of bodies; non-dimensionalize!
m = [1.989e30, 5.972e24, 3.30e23, 6.4219e23, 4.869e24]
m /= sum(m)
n = length(m)
# Gravitational constant (non-dimensionalize!)
G = 1.0

# effective force
f = u->nbody(u, G, m)

# Initial values for position and velocity properly scaled
r = [0 1 0.38 1.52 0.72 
     0 0 0    0    0 ]
v = [ zeros(1,n); 
      [ [0.0]; sqrt.(G ./ r[1,2:end]) ]'  ]

# initial condition provided in a long vector
u0 = rv2u(r, v)
;

In [ ]:
function plot_nbody(U; kwargs...)
    r, _ = u2rv(U)
    scatter(r[1,:]', r[2,:]', ms=5, xlims = (-2.5, 2.5), ylims = (-2.5, 2.5), label = ""; kwargs...)
end

h = 0.01
Ue = u0; Ui = u0 

# for n = 1:10 
while true 
    global Ue, Ui, h 
    Ue = euler_step(Ue, f, h)
    Ui = implicit_euler_step(Ui, f, h)
    sleep(0.01)
    plot( plot_nbody(Ue, title = "Explicit Euler", show=false), 
          plot_nbody(Ui, title = "Implicit Euler", show=false), size = (600, 300), show=:ijulia)
end 
IJulia.clear_output();

![](http://cse.ssl.berkeley.edu/bmendez/ay10/2002/notes/pics/bt2lfS314_a.jpg)

What is the mechanism for planets to remain in orbit? **Conservation of Energy**. For a general Hamiltonian system (I'm just ignoring masses here) we have 

$$\begin{aligned}
   \dot{q} &= p \\ 
   \dot{p} &= - \nabla V(q)
\end{aligned}$$

The *Hamiltonian* (kinetic + potential E) is 
$$ 
   H(q, p) = \frac{1}{2} |p|^2 + V(q)
$$

And $H$ is conserved along trajectories: 
$$\begin{aligned}
    \frac{d}{dt} H(q, p) &= p \cdot \dot{p} + \nabla V(q) \cdot \dot{q} \\ 
    &= - p \cdot \nabla V(q) + \nabla V(q) \cdot p = 0
\end{aligned}$$

* Because we make numerical errors it its no surprise that $H$ is not preserved along trajectories of the two Euler methods.
* But the result is more catastrophic than we might have anticipated.
* We can of course alleviate this by taking a method with higher accuracy!

In [ ]:
h = 0.1
U = u0
# @gif for time = 1:1000 
while true
    global h, U
    U = rk4_step(U, f, h)
    plot_nbody(U, title = "RK4", size = (300,300), show=:ijulia)
    sleep(0.05)
end
IJulia.clear_output();

In [ ]:
# recall the RK4 method! A fourth-order RK method, with a nice stability region!
mult_e   = z -> 1+z
mult_ie  = z -> 1 + z + z^2/2
mult_rk4 = z -> 1 + z + z^2/2 + z^3/6 + z^4/24 
plt = MATH405.levelset([-3.5,1.5], [-3.5, 3.5], [ (x, y) -> abs(mult(x + im * y)) for mult in [mult_rk4, mult_ie, mult_e] ],
                       1.0, xy=true, size = (380, 300))

### The Quadratic Case

Time for a toy model. Let's consider a linear oscillator, in fact let's consider the simplest linear oscillator, 
$$ 
\ddot{r} = - c r
$$
or equivalently 
$$\begin{aligned}
      \dot{r} &= v \\ 
      \dot{v} &= - c r
\end{aligned}$$
(We use the $c$ only to visually separate position and momentum min the equations...) The conserved Hamiltonian is 
$$ 
    H = \frac12 v^2 + \frac{c}{2} r^2.
$$
The trajectories are the ellipsi given by level-sets of $H$.

In [ ]:
harmosc(u, c=1.0) = [u[2], - c * u[1]]
u0 = [1.0, 0.0]
h = 0.1
Tf = 6 * π

Ue, te = euler_method(harmosc, u0, h, Tf)
Ui, ti = implicit_euler_method(harmosc, u0, h, Tf, nnewton=1)

plot(cos, sin, 0, 2*π, lw=5, label = "exact orbit", size = (300,300))
scatter!(Ue[1,:], Ue[2,:], label = "Euler")
plt = scatter!(Ui[1,:], Ui[2,:], label = "Implicit Euler");

So what we see is that $H(U_n) = H(R_n, V_n)$ is *increasing* for the Euler method and *decreasing* for the implicit Euler method. I.e. the two Euler methods, respectively, "create" and "dissipate" energy. With a bit of algebra this can easily be proven: 

**Proposition:** Consider the harmonic oscillator with hamiltonian $H = \frac12 v^2 + \frac{c}{2} r^2$, and let $H_n = H(U_n) = H(R_n, V_n)$ the hamiltonian value at the nth step of the Euler method, then 

$$
    H_{n+1} - H_n 
    =
    \begin{cases}
        c h^2 H_n, & \text{Euler method}  \\
        - c h^2 H_{n+1}, & \text{Implicit Euler method} 
    \end{cases}
$$

**Proof:** direct calculation; cf exercise.

Is the situation hopeless? No. We can get a lot of intuition from the theory of Hamiltonian systems, such as preservation of phase-space volume, time-reversibility, etc, and this would quite naturally lead to some new method. This would go a bit beyond this course though and so for the purposes of this lecture let us introduce an Euler method with a "bug": 

In [ ]:
"""
In this code `f` denotes the physical force and not the 
RHS in the hamiltonian dynamical system!
"""
function eulerA_step(u, f, h)   
    n = length(u) ÷ 2
    r, v = u[1:n], u[n+1:end]
    r = r + h * v 
    v = v - h * f(r)
    return rv2u(r, v)
end

eulerA_method(f, u0, h, Tf) = _iterate(f, u0, h, Tf, eulerA_step)

In [ ]:
Ua, ta = eulerA_method(r -> r, [1.0,0.0], 0.1, 6*π) 
scatter!(plt, Ua[1,:], Ua[2,:], label = "Euler-A", xlims=(-2.5, 2.5), ylims = (-2.5, 2.5));

Perfect Energy conservation! (Or is it?)

In [ ]:
He, Hi, Ha = tuple([ [ 0.5*norm(u)^2 for u in eachcol(U) ] for U in (Ue, Ui, Ua) ]...)
plot(te, He, yaxis = :log10, lw=3, label = "Explicit Euler", size = (500, 200), legend = :topleft)
plot!(ti, Hi, label = "Implicit Euler", lw=3, xlabel = "t", ylabel = "H(t)")
plot!(ta, Ha, label = "Euler-A", lw=3)

### Euler-A Method

Hamiltonian System:
$$\begin{aligned}
   \dot{q} &= p \\ 
   \dot{p} &= - \nabla V(q)
\end{aligned}$$


Euler-A method:
$$\begin{aligned}
      q_{n+1} &= q_n + h p_n \\ 
      p_{n+1} &= p_n - \nabla V(q_{n+1})
\end{aligned}$$

**Proposition:** Consider the harmonic oscillator with hamiltonian $H = \frac12 v^2 + \frac{c}{2} r^2$, and let 
$$
    H_n^A(h) = H(V_n, R_n) + \frac{c h}{2} V_n R_n
$$ 
be the *Shadow Hamiltonian* for the Euler-A method, then 
$$
    H_{n+1}^A(h) = H_n^A(h)
$$
I.e. the Shadow Hamiltonian is conserved! In particular, for $h$ sufficiently small 
$$ 
    |H_n - H_0| \leq C h.
$$
i.e. the exact Hamiltonian is conserved to within an $O(h)$ error.

**Proof:** direct calculation; cf exercise.

Let's try this on the Solar system!

In [ ]:
r = [0 1 0.38 1.52 0.72 
     0 0 0    0    0 ]
v = [ zeros(1,n); 
      [ [0.0]; sqrt.(G ./ r[1,2:end]) ]'  ]
Ua = rv2u(r, v)
h = 0.02 

Fa = r -> -nbody_accel(reshape(r, 2, :), G, m)[:]

nstep = 0
while true
# for nstep = 1:10
    nstep += 1
    Ua = eulerA_step(Ua, Fa, h)
    plot_nbody(Ua, title = "t = $(round(nstep*h, digits=1))", size = (300,300), show=:ijulia);
    sleep(0.01)
end
IJulia.clear_output();

### Outlook 

* Standard symplectic integrator: Verlet
* What we discussed today is one of the most striking and also most important examlpes of structure-preserving numerical integrators. 
* The idea is that we give up on staying globally close to a solution, but we ask that solutions are accurate for short time-intervals combined with global accuracy on physical constraints.
* Going deeper, one can often show that such schemes preserve certain statistics of the trajectories (cf. ergodicity!), and this is important in statistical sampling as well as statistical mechanics.

For more on this topic look at 

* Hairer, Ernst, Lubich, Christian, Wanner, Gerhard, Geometric Numerical Integration, Springer 2006 
* B. Leimkuhler and S. Reich, Simulating Hamiltonian Dynamics, CUP, 2004 

and getting into statistical aspects ... 

* M. Rousset, G. Stoltz and T. Lelievre, Free Energy Computations: A Mathematical Perspective, 2010
* B. Leimkuhler and C. Matthews, Molecular Dynamics, 2015 

### Molecular Dynamics

Similar to $n$-body problem, but with more complex driving force. In general *very* complex full many-body interaction. Here we will just play with a toy model.

$$
    \ddot{\bf r}_i = \sum_{j \neq i} \phi'(r_{ij}) \frac{{\bf r}_i - {\bf r}_j}{r_{ij}}
$$
where $r_{ij} = |{\bf r}_i - {\bf r}_j|$ and $\phi$ is the (non-dimensional) Lennard-Jones potential 
$$ 
    \phi(r) = r^{-12} - 2 r^{-6}
$$

In [ ]:
plot( r -> r^(-12) - 2 * r^(-6), 0.8, 3.0, size = (400, 150), lw=3, label = "", 
      xlabel = "r", xlims = [0.0, 3.0], ylabel = L"\phi(r)", ylims = [-1.2, 2.0] );

In [ ]:
function ljforce(R)
    dphi(r) = -12  * r^(-7) * (r^(-6) - 1)
    F = zeros(size(R))
    for i = 1:size(R, 2)-1, j = i+1:size(R,2)
        rij = norm(R[:,i] - R[:,j])
        Fij = dphi(rij) * (R[:,i] - R[:,j]) / rij
        F[:, i] += Fij
        F[:, j] -= Fij
    end
    return F
end

function lj_step(R, V, h)
    R += h * V
    V -= h * ljforce(R)
    return R, V    
end

lj_plot(R, ctr) = ctr > 5 ? (scatter(R[1,:], R[2,:], ms = 6, show = :ijulia, label = "", size = (400,400), xlims = [3,17], ylims = [3,17]); ctr + 1) : 0

In [ ]:
# square grid initial positions of particles
x = 0:20; o = ones(21)
X = (x * o')[:]; Y = (o * x')[:]
R = [ X'; Y' ]
# random initial velocities 
V = 0.01 * randn(size(R))
h = 0.03

default(show=:ijulia)
ctr = lj_plot(R, 100)

while true
# for nstep = 1:10
    R, V = lj_step(R, V, h)
    ctr = lj_plot(R, ctr)
    sleep(0.01)
end
IJulia.clear_output();

In [ ]:
using PyCall 
YouTubeVideo = pyimport("IPython.lib.display").YouTubeVideo
YouTubeVideo("_0QhOCDzP4I")  # salt crystal melting in water